In [ ]:
# ==============================================
# Bipartite Network Analysis: Users & Spaces
# ==============================================

import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
from google.colab import files
import io
import numpy as np

# ==============================================
# PART 1: UPLOAD DATA FILES
# ==============================================
print("Upload 'sample_user_nodes.csv', 'sample_space_nodes.csv', and 'sample_edges.csv'")

uploaded = files.upload()
user_nodes = pd.read_csv(io.BytesIO(uploaded['sample_user_nodes.csv']))
space_nodes = pd.read_csv(io.BytesIO(uploaded['sample_space_nodes.csv']))
edges = pd.read_csv(io.BytesIO(uploaded['sample_edges.csv']))

print(f"Users loaded: {user_nodes.shape[0]}")
print(f"Spaces loaded: {space_nodes.shape[0]}")
print(f"Edges loaded: {edges.shape[0]}")

# ==============================================
# PART 2: BUILD BIPARTITE GRAPH
# ==============================================
G = nx.Graph()

# Add user nodes
user_set = set(user_nodes['id'])
G.add_nodes_from(user_set, bipartite='User')

# Add space nodes
space_set = set(space_nodes['id'])
G.add_nodes_from(space_set, bipartite='Space')

# Add edges (user-space relationships)
for _, row in edges.iterrows():
    G.add_edge(row['source'], row['target'])

print(f"\nGraph successfully created!")
print(f"Total nodes: {G.number_of_nodes()} | Total edges: {G.number_of_edges()}")

# ==============================================
# PART 3: CHECK IF GRAPH IS BIPARTITE
# ==============================================
if not bipartite.is_bipartite(G):
    raise ValueError("The graph is not bipartite. Please check your input files.")

# ==============================================
# PART 4: CALCULATE NETWORK METRICS
# ==============================================
print("\nCalculating bipartite network measures...\n")

# ---- 1. Number of nodes ----
num_users = len(user_set)
num_spaces = len(space_set)
total_nodes = G.number_of_nodes()
print(f"Number of nodes: {total_nodes} ({num_users} Users, {num_spaces} Spaces)")

# ---- 2. Number of edges ----
num_edges = G.number_of_edges()
print(f"Number of edges: {num_edges}")

# ---- 3. Density ----
density = bipartite.density(G, user_set)
print(f"Density: {density:.6f}")

# ---- 4. Number of components ----
num_components = nx.number_connected_components(G)
print(f"Number of components: {num_components}")

# ---- 5. Clustering coefficients ----
# Mean clustering for Users and Spaces separately (bipartite formula)
clustering_users = bipartite.average_clustering(G, user_set)
clustering_spaces = bipartite.average_clustering(G, space_set)

# Overall clustering using bipartite dot-product formula
clustering_overall = np.mean(list(bipartite.clustering(G, mode='dot').values()))

print(f"Mean clustering coefficient (Users): {clustering_users:.4f}")
print(f"Mean clustering coefficient (Spaces): {clustering_spaces:.4f}")
print(f"Mean clustering coefficient (Overall): {clustering_overall:.4f}")

# ---- 6. Mean degree (Users & Spaces) ----
mean_degree_users = np.mean([G.degree(u) for u in user_set])
mean_degree_spaces = np.mean([G.degree(s) for s in space_set])
print(f"Mean degree (Users): {mean_degree_users:.3f}")
print(f"Mean degree (Spaces): {mean_degree_spaces:.3f}")


